<h1>MODEL EVALUATOR</h1>

In [ ]:
# LOAD DEPENDENCIES
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import itertools, scikitplot
import os, cv2, glob, time, pickle, logging

from keras_flops import get_flops
from PIL import Image, ImageDraw, ImageFont
from keras.utils.layer_utils import count_params
from matplotlib.font_manager import FontProperties
from tensorflow.keras.models import Model, load_model
from matplotlib.ticker import FuncFormatter, PercentFormatter
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import (accuracy_score, mean_squared_error, 
                             mean_squared_log_error, classification_report, 
                             confusion_matrix, roc_curve, auc)

# PREVENT ERROR UNCESSARY MESSAGES
tf.get_logger().setLevel(logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
# LOAD THE DATA
cwd = os.getcwd()
main_ds = os.path.join(cwd, 'ds')

task = ['binary', 'severity']
ds_folder = ['ua', 'torgo']

task_idx, ds_folder_idx = 1, 1  # Change to select binary or severity task, and 'torgo' dataset (0 for 'ua' dataset)

selected_ds = os.path.join(main_ds, ds_folder[ds_folder_idx], f"{ds_folder[ds_folder_idx]}_{task[task_idx]}")
print(f"{selected_ds=}")

train_data_dir = os.path.join(selected_ds, 'train')
validation_data_dir = os.path.join(selected_ds, 'val')
test_data_dir = os.path.join(selected_ds, 'test')

print("*" * 55)
print("Data folders found!")
print("*" * 55)
print(f"Train Path: {train_data_dir}")
print(f"Validation Path: {validation_data_dir}")
print(f"Test Path: {test_data_dir}")
print("*" * 55)

In [ ]:
architecture_name = "DySARNet"
model_name = f"{architecture_name}_{task[task_idx]}_{ds_folder[ds_folder_idx]}"
model_path = os.path.join('model/', model_name)

print(f"Model name: {model_name}")

In [ ]:
# Set parameters
batch_size = 8
img_rows, img_cols = 224, 224
preprocessing = ImageDataGenerator(rescale = 1. / 255)

In [ ]:
# DATA GENERATORS
if task_idx == 0:
    classes = ['0_Normal', '1_Dysarthria']
    labels = ['Normal', 'Dysarthria']
else:
    classes = ['0_Normal', '1_Mild', '2_Moderate', '3_SeverelyModerate', '4_Severe']
    labels = ['Normal', 'Mild', 'Moderate', 'Severly Moderate', 'Severe']

val_datagen = preprocessing

test_datagen = preprocessing

validation_generator = val_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows,img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        seed=42,
        shuffle=False,
        classes=classes)

test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_rows,img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        seed=42,
        shuffle=False,
        classes=classes)

# CHECK  THE NUMBER OF SAMPLES
nb_validation_samples = len(validation_generator.filenames)
nb_test_samples = len(test_generator.filenames)

print("Validation samples:", nb_validation_samples)
print("Test samples:", nb_test_samples)
print(f"TOTAL SAMPLES: {(nb_validation_samples + nb_test_samples)}")

if nb_validation_samples == 0:
    print("NO DATA VALIDATION FOUND! Please check your validation data path and folders!")
    print("Check the data folders first!")
else:
    print("Validation samples found!")
    
if nb_test_samples == 0:
    print("NO DATA TEST FOUND! Please check your test data path and folders!")
    print("Check the data folders first!")
else:
    print("Test samples found!")

# check the class indices
validation_generator.class_indices
test_generator.class_indices

# true labels
Y_test=validation_generator.classes
test_labels = test_generator.classes

num_classes= len(validation_generator.class_indices)

if nb_validation_samples and nb_test_samples > 0:
    print("Generators are set!")
    print("Check if dataset is complete and has no problems before proceeding.")

In [ ]:
model_directory = 'pretrained_models'
model_path = os.path.join(model_directory, model_name)

model_file_pattern = os.path.join(model_path, f'{model_name}-*.h5')
history_file_pattern = os.path.join(model_path, f'{model_name}-*.history')

In [ ]:
# Find all model files and history files matching the pattern
model_files = glob.glob(model_file_pattern)
history_files = glob.glob(history_file_pattern)
print(f"Searching for models at path: {model_file_pattern}")

if len(model_files) == 0:
    print(f"No models found for {model_name}")
else:
    for model_file in model_files:
        # Load the model
        model = load_model(model_file)
        loaded_file = os.path.basename(model_file)
        print(f"The model {loaded_file} is loaded")
        
if len(history_files) == 0:
    print(f"No history files found for {model_name}")
else:
    for history_file in history_files:
        # Load the history file
        with open(history_file, 'rb') as f:
            history = pickle.load(f)
        loaded_history = os.path.basename(history_file)
        print(f"The history {loaded_history} is loaded")

In [ ]:
# Measure the Cost-efficiency via FLOPS.
flops  = float("{0:.2f}".format(get_flops(Model(model.input, model.output), batch_size=1)/ 10 ** 9))
params = float("{0:.2f}".format(model.count_params() / 10 ** 6))
trainable_count = float("{0:.2f}".format(count_params(model.trainable_weights) / 10 ** 6))
print(f"{model_name} FLOPS and Parameters:")
print('-'*25)
print("FLOPS:", flops, "GFLOPS")
print("Params:", params, "M")
print('-'*25)

In [ ]:
# Validate the model performance.
print(f"{model_name} is Validating....\n")
start_time = time.time()

val_scores = model.evaluate(validation_generator, return_dict=True, verbose=1)

val_acc = val_scores['accuracy'] * 100
val_loss = val_scores['loss'] * 100

print(f"Val accuracy: {val_acc:.2f}%")
print(f"Val loss: {val_loss:.2f}%")

elapsed_time = time.time() - start_time
val_time = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))

print("\nValidation Time:")
print("-"*15)
print(f"Elapsed time in seconds: {elapsed_time:.2f} seconds")
print(f"Elapsed time in minutes: {elapsed_time / 60:.2f} minutes")
print(f"Elapsed time in hours: {elapsed_time / 3600:.2f} hours")
print()
print(f"Total Validation Time: {val_time}")

In [ ]:
# Test the model performance.
print(f"{model_name} is Testing....\n")
start_time = time.time()

test_scores = model.evaluate(test_generator, return_dict=True, verbose=1)

test_acc = test_scores['accuracy'] * 100
test_loss = test_scores['loss'] * 100

print(f"Test accuracy: {test_acc:.2f}%")
print(f"Test loss: {test_loss:.2f}%")

elapsed_time = time.time() - start_time
test_time = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))

print("\nTest Time:")
print("-"*15)
print(f"Elapsed time in seconds: {elapsed_time:.2f} seconds")
print(f"Elapsed time in minutes: {elapsed_time / 60:.2f} minutes")
print(f"Elapsed time in hours: {elapsed_time / 3600:.2f} hours")
print()
print(f"Total Test Time: {test_time}")

In [ ]:
# Inference the model performance with validation data.
print(f"{model_name} is Inferencing with the Validation Data....\n")
start_time = time.time()

y_pred = model.predict(validation_generator, 
                       nb_validation_samples // batch_size, 
                       workers=1, 
                       verbose=1)

elapsed_time = time.time() - start_time
val_inference_time = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))

print("\nValidation Inference Time:")
print("-"*15)
print(f"Elapsed time in seconds: {elapsed_time:.2f} seconds")
print(f"Elapsed time in minutes: {elapsed_time / 60:.2f} minutes")
print(f"Elapsed time in hours: {elapsed_time / 3600:.2f} hours")
print()
print(f"Total Test Time: {val_inference_time}")

In [ ]:
# Inference the model performance with test data.
print(f"{model_name} is Inferencing with the Test Data....\n")

start_time = time.time()

test_pred = model.predict(test_generator, 
                          nb_validation_samples/batch_size, 
                          workers=1, 
                          verbose=1)

elapsed_time = time.time() - start_time
test_inference_time = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))

print("\nTest Inference Time:")
print("-"*15)
print(f"Elapsed time in seconds: {elapsed_time:.2f} seconds")
print(f"Elapsed time in minutes: {elapsed_time / 60:.2f} minutes")
print(f"Elapsed time in hours: {elapsed_time / 3600:.2f} hours")
print()
print(f"Total Test Time: {test_inference_time}")

In [ ]:
# Take the test accuracy percentage to name the model

test_acc = test_scores['accuracy']
accuracy_percentage = "{:.2%}".format(test_acc)

print(f'The model {model_name} achieved a test accuracy of: {accuracy_percentage}')

In [ ]:
# Set the figure save point.

# Define the figure save point.
fig_save_point = f'figures/{task[task_idx].title()}/{ds_folder[ds_folder_idx].upper()}/{model_name}/'

# Check if the directory exists, create it if not.
if not os.path.exists(fig_save_point):
    os.makedirs(fig_save_point)
    print(f"Directory '{fig_save_point}' created successfully.")
else:
    print(f"Directory '{fig_save_point}' already exists, no new folder is created.")

In [ ]:
# Set the image saving format and DPI.
dpi = 300
fformat = 'svg'

In [ ]:
# Create figure and subplot with specified size
fig, ax = plt.subplots(figsize=(4, 4))

# Loss Curves
ax.plot(history['loss'], '#0F52BA', linewidth=3.0, marker='8')
ax.plot(history['val_loss'], '#800020', linewidth=3.0, ls='--')

# Accuracy Curves
ax.plot(history['accuracy'], '#006D77', linewidth=3.0, marker='8')
ax.plot(history['val_accuracy'], '#C28800', linewidth=3.0, ls='--')

# Add shaded area for loss error
loss_error = np.std(history['val_loss'])
ax.fill_between(range(len(history['val_loss'])), 
                history['val_loss']-loss_error, 
                history['val_loss']+loss_error, color='#800020', alpha=0.2)

# Add shaded area for accuracy error
acc_error = np.std(history['val_accuracy'])
ax.fill_between(range(len(history['val_accuracy'])), 
                history['val_accuracy']-acc_error, 
                history['val_accuracy']+acc_error, 
                color='#C28800', 
                alpha=0.2)

# Set labels, titles, and grid
ax.set_xlabel('Epochs', fontsize=16)
ax.set_ylabel('Accuracy/Loss', fontsize=16)
ax.grid(True, linewidth=1)
ax.tick_params(width=2)
# ax.set_title('Accuracy and Loss Curves', fontsize=16)

# Format y-axis labels as percentages
ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.0%}'.format(y)))

# Set legend and linewidths
ax.legend(['Train Loss', 'Val Loss', 'Train Acc', 'Val Acc'], fontsize=8, loc='best')
for spine in ax.spines.values():
    spine.set_linewidth(2)

plt.tight_layout()
plt.savefig(fig_save_point + '0-' + model_name + '_' + accuracy_percentage + '-' + 'combined_loss_curves.svg', format=fformat, dpi=dpi)

plt.show()

In [ ]:
# Create figure and subplots with specified size
fig, ax = plt.subplots(1, 2, figsize=(10, 4))

# Loss Curves
ax[0].plot(history['loss'],'#ff7f0e',linewidth=3.0, marker='8')
ax[0].plot(history['val_loss'],'#1f77b4',linewidth=3.0, ls='--')

# Add shaded area for loss error
loss_error = np.std(history['val_loss'])
ax[0].fill_between(range(len(history['val_loss'])), 
                   history['val_loss']-loss_error, 
                   history['val_loss']+loss_error, color='#ff7f0e', alpha=0.2)


ax[0].legend(['Training loss', 'Validation Loss'],fontsize=12)
ax[0].set_xlabel('Epochs ',fontsize=10)
ax[0].set_ylabel('Loss',fontsize=10)
ax[0].set_title('Loss Curves',fontsize=12)
ax[0].grid(True, linewidth=1)
ax[0].tick_params(width=2)

for spine in ax[0].spines.values():
    spine.set_linewidth(2)

# Accuracy Curves
ax[1].plot(history['accuracy'],'#ff7f0e',linewidth=3.0, marker='8')
ax[1].plot(history['val_accuracy'],'#1f77b4',linewidth=3.0, ls='--')

# Add shaded area for accuracy error
acc_error = np.std(history['val_accuracy'])
ax[1].fill_between(range(len(history['val_accuracy'])), 
                   history['val_accuracy']-acc_error, 
                   history['val_accuracy']+acc_error, 
                   color='#ff7f0e', 
                   alpha=0.2)

ax[1].legend(['Training Accuracy', 'Validation Accuracy'],fontsize=12)
ax[1].set_xlabel('Epochs',fontsize=10)
ax[1].set_ylabel('Accuracy',fontsize=10)
ax[1].set_title('Accuracy Curves',fontsize=12)
ax[1].grid(True, linewidth=1)
ax[1].tick_params(width=2)
ax[1].yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.0%}'.format(y)))

for spine in ax[1].spines.values():
    spine.set_linewidth(2)

plt.tight_layout()
# fig.savefig(fig_save_point + model_name + '_' + accuracy_percentage + '_' + 'loss_acc_curves.svg', format='svg')
plt.show()

In [ ]:
#The confusion matrix method
from mpl_toolkits.axes_grid1 import make_axes_locatable

if len(labels) == 5:
    labels = ['0', '1', '2', '3', '4']
else:
    labels = ['0', '1']

def plot_confusion_matrix(cm, fontsize=16, classes=labels, normalize=True, title=model._name + ' Validation', cmap=plt.cm.bone, ax=None, filename=None):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    
    if ax is None:
        fig, ax = plt.subplots()
    
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.set_title(title)
    
    # Dynamically adjust the height of the colorbar based on the height of the confusion matrix plot
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05*(10./len(classes)))
    ax.figure.colorbar(im, cax=cax)
    cbar = ax.figure.colorbar(im, cax=cax)
    cbar.ax.yaxis.set_label_coords(1.8, 0.5)
    
    # Change the font size of the color bar
    cbar.ax.tick_params(labelsize=12, rotation=90) 
    
    tick_marks = np.arange(len(classes))
    ax.set_xticks(tick_marks)
    ax.set_yticks(tick_marks)
    ax.set_xticklabels(classes, rotation=0, ha='center', fontsize=fontsize)
    ax.set_yticklabels(classes, rotation=0, ha='center', fontsize=fontsize)
#     ax.tick_params(axis='both', which='major', labelsize=12, pad=5)
    
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        ax.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="black" if cm[i, j] > thresh else "white", 
                 fontsize=fontsize, ha='center')
    
    ax.set_ylabel('True label', fontsize=fontsize)
    ax.set_xlabel('Predicted label', fontsize=fontsize)
    ax.grid(False)
    
    if filename:
        fig.savefig(filename, format='svg')
    
    return ax

In [ ]:
# Print the validation results using a confusion matrix.

target_names = classes
print(classification_report(Y_test,y_pred.argmax(axis=-1),
                            target_names=target_names, 
                            digits=4))

cm_validation = confusion_matrix(Y_test, y_pred.argmax(axis=-1))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,6))

val_cm_plot = plot_confusion_matrix(cm_validation, 
                                    classes=labels,
                                    normalize=False,
                                    title='',
                                    ax=ax1)

val_cm_plot_normalized = plot_confusion_matrix(cm_validation, 
                                               classes=labels,
                                               normalize=True,
                                               title='',
                                               ax=ax2)

# fig.savefig(fig_save_point + model_name + '_' + accuracy_percentage + '_' + 'val_cm_both.svg', format='svg')

fig, ax1 = plt.subplots(1, 1, figsize=(6,6))

val_cm_plot = plot_confusion_matrix(cm_validation, 
                                    classes=labels,
                                    normalize=False,
                                    title='',
                                    ax=ax1)

fig.savefig(fig_save_point + '1-' + model_name + '_' + accuracy_percentage + '-' + 'val_cm.svg', format=fformat, dpi=dpi)

fig, ax1 = plt.subplots(1, 1, figsize=(6,6))

val_cm_plot_normalized = plot_confusion_matrix(cm_validation, 
                                               classes=labels, 
                                               normalize=True,
                                               title='',
                                               ax=ax1)

fig.savefig(fig_save_point + '1-' + model_name + '_' + accuracy_percentage + '-' + 'val_cm_norm.svg', format=fformat, dpi=dpi)


In [ ]:
# Print the test results using a confusion matrix
print(classification_report(test_labels,test_pred.argmax(axis=-1),
                            target_names=classes, 
                            digits=4))

cm_test = confusion_matrix(test_labels, test_pred.argmax(axis=-1))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,6))

test_cm_plot = plot_confusion_matrix(cm_test, 
                                    classes=labels, 
                                    normalize=False,
                                    title='', 
                                    ax=ax1)

test_cm_plot_normalized = plot_confusion_matrix(cm_test, 
                                               classes=labels,
                                               normalize=True,
                                               title='', 
                                               ax=ax2)

fig, ax1 = plt.subplots(1, 1, figsize=(6,6))

test_cm_plot = plot_confusion_matrix(cm_test, 
                                    classes=labels, 
                                    normalize=False, 
                                    title='', 
                                    ax=ax1)

fig.savefig(fig_save_point + '2-' + model_name + '_' + accuracy_percentage + '-' + 'test_cm.svg', format=fformat, dpi=dpi)

fig, ax1 = plt.subplots(1, 1, figsize=(6,6))

test_cm_plot_normalized = plot_confusion_matrix(cm_test, 
                                               classes=labels, 
                                               title='',
                                               normalize=True,
                                               ax=ax1)

fig.savefig(fig_save_point + '2-' + model_name + '_' + accuracy_percentage + '-' + 'test_cm_norm.svg', format=fformat, dpi=dpi)

In [ ]:
# Generate the ROC and PR curves for the val data.

# Set font properties
font = FontProperties()
font.set_family('Tahoma')
font.set_size(18)

# Plot ROC curve for validation set
scikitplot.metrics.plot_roc(Y_test, y_pred)
# plt.title(model._name + ' Validation ROC Curve', fontproperties=font)
plt.title('')
plt.xlabel('Specificity', fontproperties=font)
plt.ylabel('Sensitivity', fontproperties=font)
plt.legend(fontsize=12)
plt.tight_layout()
plt.savefig(fig_save_point + '3-' + model_name + '_' + accuracy_percentage + '-' + 'roc_val.svg', format=fformat, dpi=dpi)
plt.show()

# Plot precision-recall curve for validation set
scikitplot.metrics.plot_precision_recall(Y_test, y_pred)
# plt.title(model._name + ' Validation Precision-Recall Curve', fontproperties=font)
plt.title('')
plt.xlabel('Recall', fontproperties=font)
plt.ylabel('Precision', fontproperties=font)
plt.legend(fontsize=12)
plt.tight_layout()
plt.savefig(fig_save_point + '3-' + model_name + '_' + accuracy_percentage + '-' + 'pr_val.svg', format=fformat, dpi=dpi)
plt.show()

In [ ]:
# Generate the ROC and PR curves for the test data.

# Plot ROC curve for test set
scikitplot.metrics.plot_roc(test_labels, test_pred)
# plt.title(model._name + ' Test ROC Curve', fontproperties=font)
plt.title('')
plt.xlabel('Specificity', fontproperties=font)
plt.ylabel('Sensitivity', fontproperties=font)
plt.legend(fontsize=12)
plt.tight_layout()
plt.savefig(fig_save_point + '4-' + model_name + '_' + accuracy_percentage + '-' + 'roc_test.svg', format=fformat, dpi=dpi)
plt.show()

# Plot precision-recall curve for test set
scikitplot.metrics.plot_precision_recall(test_labels, test_pred)
# plt.title(model._name + ' Test Precision-Recall Curve', fontproperties=font)
plt.title('')
plt.xlabel('Recall', fontproperties=font)
plt.ylabel('Precision', fontproperties=font)
plt.legend(fontsize=12)
plt.tight_layout()
plt.savefig(fig_save_point + '4-' + model_name + '_' + accuracy_percentage + '-' + 'pr_test.svg', format=fformat, dpi=dpi)

plt.show()

In [ ]:
# Summarize the overall scores.
precision = np.zeros(cm_test.shape[0])
recall = np.zeros(cm_test.shape[0])
f1 = np.zeros(cm_test.shape[0])

for i in range(cm_test.shape[0]):
    col_sum = np.sum(cm_test[:, i])
    if col_sum == 0:
        precision[i] = 0
        recall[i] = 0
        f1[i] = 0
    else:
        precision[i] = cm_test[i, i] / col_sum
        recall[i] = cm_test[i, i] / np.sum(cm_test[i, :])
        f1[i] = 2 * (precision[i] * recall[i]) / (precision[i] + recall[i])

precision_mean = np.mean(precision)
recall_mean = np.mean(recall)
f1_mean = np.mean(f1)
test_acc = test_scores['accuracy']

# Format the precision, recall, and f1-score values as percentages with 2 decimal points
precision_mean_percentage = "{:.2%}".format(precision_mean)
recall_mean_percentage = "{:.2%}".format(recall_mean)
f1_mean_percentage = "{:.2%}".format(f1_mean)
accuracy_percentage = "{:.2%}".format(test_acc)

print(f'Test Scores for {model_name}:')
print('-'*len(model_name)*2)
print(f'Accuracy: {accuracy_percentage}')
print(f'Precision: {precision_mean_percentage}')
print(f'Recall: {recall_mean_percentage}')
print(f'F1-score: {f1_mean_percentage}')
print(f'FLOPS: {flops} GFlops')
print(f'Parameters: {params} M')
print('-'*len(model_name)*2)

def check_if_content_exists(file_path, content):
    with open(file_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            if content in line:
                return True
    return False

# Save the output to a text file
with open('test_scores.txt', 'a') as f:
    if not check_if_content_exists('test_scores.txt', f'Test Scores for {model_name}:\n'):
        f.write(f'Test Scores for {model_name}:\n')
        f.write('-'*len(model_name)*2 + '\n')
        f.write(f'Accuracy: {accuracy_percentage}\n') 
        f.write(f'Precision: {precision_mean_percentage}\n')
        f.write(f'Recall: {recall_mean_percentage}\n')
        f.write(f'F1-score: {f1_mean_percentage}\n')
        f.write(f'FLOPS: {flops} GFLOPs\n')
        f.write(f'Parameters: {params} M\n')
        f.write('-'*len(model_name)*2 + '\n')
        f.write('\n\n')

# Load the text file and convert it to an image
with open('test_scores.txt', 'r') as f:
    text = f.read()

font = ImageFont.truetype('arial.ttf', 16)
img = Image.new('RGB', (500, 200), color=(255, 255, 255))
draw = ImageDraw.Draw(img)
draw.text((20, 20), text, font=font, fill=(0, 0, 0))

# Save the image as a PDF
img.save(fig_save_point + '5-' + model_name + '_' + accuracy_percentage + '-' + 'summary.pdf', format='PDF', dpi=(300,300))

In [ ]:
# Create figure and subplots with specified size
fig, ax = plt.subplots(2, 1, figsize=(10, 10))

# Loss Curves
ax[0].plot(history['loss'],'#ff7f0e',linewidth=3.0)
ax[0].plot(history['val_loss'],'#1f77b4',linewidth=3.0)

# Add shaded area for loss error
loss_error = np.std(history['val_loss'])
ax[0].fill_between(range(len(history['val_loss'])), 
                   history['val_loss']-loss_error, 
                   history['val_loss']+loss_error, 
                   color='#ff7f0e', 
                   alpha=0.2)

# Add markers for lowest loss
min_loss_index = history['val_loss'].index(min(history['val_loss']))
ax[0].scatter(min_loss_index, 
              min(history['val_loss']), 
              c='#2ca02c', 
              s=100)

ax[0].legend(['Training loss', 'Validation Loss'],fontsize=12)
ax[0].set_xlabel('Epochs ',fontsize=10)
ax[0].set_ylabel('Loss',fontsize=10)
ax[0].set_title('Loss Curves',fontsize=12)
ax[0].grid(True, linewidth=1)
ax[0].tick_params(width=2)

for spine in ax[0].spines.values():
    spine.set_linewidth(2)

# Accuracy Curves
ax[1].plot(history['accuracy'],'#ff7f0e',linewidth=3.0)
ax[1].plot(history['val_accuracy'],'#1f77b4',linewidth=3.0)

# Add shaded area for accuracy error
acc_error = np.std(history['val_accuracy'])
ax[1].fill_between(range(len(history['val_accuracy'])), 
                   history['val_accuracy']-acc_error, 
                   history['val_accuracy']+acc_error, 
                   color='#ff7f0e', 
                   alpha=0.2)

# Add markers for highest accuracy
max_acc_index = history['val_accuracy'].index(max(history['val_accuracy']))
ax[1].scatter(max_acc_index, max(history['val_accuracy']), c='#2ca02c', s=100)

# Annotate the highest accuracy
max_acc_epoch = max_acc_index + 1  # Add 1 because epoch numbering starts at 0
ax[1].annotate(f'Highest accuracy: {max(history["val_accuracy"]):.2%} (epoch {max_acc_epoch})',
             xy=(max_acc_index, max(history["val_accuracy"])),
             xytext=(max_acc_index+10, max(history["val_accuracy"])-0.05),
             arrowprops=dict(facecolor='black', shrink=0.05))

ax[1].legend(['Training Accuracy', 'Validation Accuracy'],fontsize=12)
ax[1].set_xlabel('Epochs',fontsize=10)
ax[1].set_ylabel('Accuracy',fontsize=10)
ax[1].set_title('Accuracy Curves',fontsize=12)
ax[1].grid(True, linewidth=1)
ax[1].tick_params(width=2)
ax[1].yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.0%}'.format(y)))

# Annotate the lowest loss
min_loss_index = history['val_loss'].index(min(history['val_loss']))
ax[0].annotate(f'Lowest loss: {min(history["val_loss"]):.2%} (epoch {min_loss_index+1})',
             xy=(min_loss_index, min(history["val_loss"])),
             xytext=(min_loss_index+10, min(history["val_loss"])-0.05),
             arrowprops=dict(facecolor='black', shrink=0.05))

for spine in ax[1].spines.values():
    spine.set_linewidth(2)

plt.tight_layout()
plt.show()
